In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../bin/data_preprocess.csv')

In [4]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
31,co2,231,59.175045,59.175045,float64
16,consumo_carretera,91,56.393874,56.393874,float64
17,consumo_urbano,165,56.384478,56.384478,float64
15,consumo_medio,129,51.329512,51.329512,float64
33,id_sobrealimentacion,4,43.866078,43.866078,object
18,deposito,86,36.033073,36.033073,float64
20,cilindrada,226,36.004886,36.004886,float64
32,num_cilindros,8,35.738670,35.738670,float64
27,velocidad_max,155,30.420621,30.420621,float64
10,capacidad_maletero,386,30.276551,30.276551,float64


In [5]:
comunidades_dict = {
    "Andalucía": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"],
    "Aragón": ["Huesca", "Teruel", "Zaragoza"],
    "Asturias": ["Asturias"],
    "Islas Baleares": ["Islas Baleares"],
    "Canarias": ["Las Palmas", "Tenerife"],
    "Cantabria": ["Cantabria"],
    "Castilla-La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
    "Castilla y León": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
    "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
    "Extremadura": ["Badajoz", "Cáceres"],
    "Galicia": ["La Coruña", "Lugo", "Orense", "Pontevedra"],
    "Madrid": ["Madrid"],
    "Murcia": ["Murcia"],
    "Navarra": ["Navarra"],
    "La Rioja": ["La Rioja"],
    "País Vasco": ["Álava", "Guipúzcoa", "Vizcaya"],
    "Valencia": ["Alicante", "Castellón", "Valencia"],
    "Ceuta": ["Ceuta"],
    "Melilla": ["Melilla"]
}

In [6]:
provincias_to_comunidades = {
    provincia: comunidad
    for comunidad, provincias in comunidades_dict.items()
    for provincia in provincias
}

In [7]:
listado_provincias = {}
index = 1
for detalle in df['id_provincia'].unique():
    if pd.notnull(detalle): 
        listado_provincias[index] = detalle
        index += 1

df_provincias = pd.DataFrame.from_dict(listado_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_provincias['comunidad_autonoma'] = df_provincias['nombre_provincia'].map(provincias_to_comunidades)
df_provincias.to_csv('../bin/listado_provincias.csv', index=False)

conversion_provincias = {v: k for k, v in listado_provincias.items()}
df['id_provincia'] = df['id_provincia'].map(conversion_provincias)

In [8]:
listado_concesionarios = {}
index = 1
for detalle in df['id_concesionario'].unique():
    if pd.notnull(detalle): 
        listado_concesionarios[index] = detalle
        index += 1

df_concesionarios = pd.DataFrame.from_dict(listado_concesionarios, orient='index', columns=['nombre_concesionario']).reset_index()
df_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)
df_concesionarios.to_csv('../bin/listado_concesionarios.csv', index=False)

conversion = {v: k for k, v in listado_concesionarios.items()}
df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [9]:
listado_modelos = {}
index = 1
for detalle in df['id_modelo'].unique():
    if pd.notnull(detalle): 
        listado_modelos[index] = detalle
        index += 1

df_modelos = pd.DataFrame.from_dict(listado_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)
df_modelos.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_modelos.items()}
df['id_modelo'] = df['id_modelo'].map(conversion)

In [10]:
marca_df = pd.read_csv('../bin/listado_marcas.csv')

marca_df['nombre_marca'] = marca_df['nombre_marca'].str.upper()
listado_marcas = marca_df.set_index('id_marca')['nombre_marca'].to_dict()

conversion = {v: k for k, v in listado_marcas.items()}
df['id_marca'] = df['id_marca'].map(conversion)

In [11]:
listado_distintivo_ambiental = {}
index = 1
for detalle in df['id_distintivo_ambiental'].unique():
    if pd.notnull(detalle): 
        listado_distintivo_ambiental[index] = detalle
        index += 1
    
df_distintivo_ambiental = pd.DataFrame.from_dict(listado_distintivo_ambiental, orient='index', columns=['nombre_distintivo']).reset_index()
df_distintivo_ambiental.rename(columns={'index': 'id_distintivo'}, inplace=True)
df_distintivo_ambiental.to_csv('../bin/listado_distintivo_ambiental.csv', index=False)

conversion = {v: k for k, v in listado_distintivo_ambiental.items()}
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [12]:
listado_combustibles = {}
index = 1
for detalle in df['combustible'].unique():
    if pd.notnull(detalle): 
        listado_combustibles[index] = detalle
        index += 1
    
df_combustibles = pd.DataFrame.from_dict(listado_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)
df_combustibles.to_csv('../bin/listado_combustibles.csv', index=False)

conversion = {v: k for k, v in listado_combustibles.items()}
df['id_combustible'] = df['combustible'].map(conversion)

df = df.drop(columns=['combustible'])

In [13]:
listado_tipos_cambio = {}
index = 1
for detalle in df['tipo_cambio'].unique():
    if pd.notnull(detalle): 
        listado_tipos_cambio[index] = detalle
        index += 1
    
df_tipos_cambio = pd.DataFrame.from_dict(listado_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)
df_tipos_cambio.to_csv('../bin/listado_tipos_cambio.csv', index=False)

tipos_cambio_to_indices = {v: k for k, v in listado_tipos_cambio.items()}
df['id_tipo_cambio'] = df['tipo_cambio'].map(tipos_cambio_to_indices)

df = df.drop(columns=['tipo_cambio'])

In [14]:
listado_sobrealimentacion = {}
index = 1
for detalle in df['id_sobrealimentacion'].unique():
    if pd.notnull(detalle): 
        listado_sobrealimentacion[index] = detalle
        index += 1
    
df_sobrealimentacion = pd.DataFrame.from_dict(listado_sobrealimentacion, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_sobrealimentacion.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)
df_sobrealimentacion.to_csv('../bin/listado_sobrealimentacion.csv', index=False)

conversion = {v: k for k, v in listado_sobrealimentacion.items()}
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [15]:
listado_traccion = {}
index = 1
for detalle in df['id_traccion'].unique():
    if pd.notnull(detalle): 
        listado_traccion[index] = detalle
        index += 1
    
df_traccion = pd.DataFrame.from_dict(listado_traccion, orient='index', columns=['nombre_traccion']).reset_index()
df_traccion.rename(columns={'index': 'id_traccion'}, inplace=True)
df_traccion.to_csv('../bin/listado_traccion.csv', index=False)

conversion = {v: k for k, v in listado_traccion.items()}
df['id_traccion'] = df['id_traccion'].map(conversion)

In [16]:
df.head()

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,cilindrada,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo,id_combustible,id_tipo_cambio
0,1.0,58000.0,1,1.0,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,480.0,5.0,2810.0,2045.0,4.0,4.5,4.0,5.3,57.0,1995.0,1.0,6.0,110.0,150.0,320.0,215.0,8.7,11,2018,118.0,4.0,1.0,8.0,1.0,1.0,1
1,1.0,34680.0,1,2.0,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,185.0,4.0,2300.0,1055.0,3.0,NaN,NaN,NaN,35.0,NaN,2.0,6.0,52.0,70.0,92.0,167.0,13.8,12,2021,NaN,3.0,2.0,19.0,2.0,2.0,1
2,1.0,66933.0,2,1.0,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,430.0,5.0,2646.0,1910.0,5.0,3.8,3.6,4.2,55.0,1461.0,2.0,6.0,81.0,110.0,260.0,182.0,11.9,4,2017,103.0,4.0,2.0,50.0,3.0,1.0,1
3,2.0,159000.0,3,1.0,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,540.0,5.0,2662.0,1985.0,5.0,3.6,3.3,4.2,48.0,1598.0,2.0,6.0,81.0,110.0,300.0,195.0,11.4,1,2017,99.0,4.0,2.0,52.0,4.0,1.0,1
4,1.0,62695.0,4,3.0,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,404.0,5.0,2670.0,2425.0,5.0,NaN,NaN,NaN,43.0,2360.0,3.0,1.0,138.0,188.0,193.0,162.0,10.9,7,2022,NaN,4.0,2.0,48.0,5.0,3.0,2


In [32]:
df.to_csv('../bin/prestaciones.csv', index=False)

In [ ]:
df_links = pd.read_pickle("anuncios_coches_imagenes.pkl")
df_links = df_links.reset_index(drop=True)
df_links.index += 1
df_links["id"] = df_links.index
df_links = df_links[["id", "link_anuncio", "link_foto"]]